In [1]:
import pandas as pd
from pyArango.connection import *
from pyArango.collection import *

In [2]:
conn = Connection(username="ngph@ngph", password="ngph")

In [3]:
df = pd.read_csv('../assets/data/transfer_network_201812.csv')

In [6]:
db = conn["m2"]

In [7]:
users=None
if not db.hasCollection('users'):
    users = db.createCollection(name="users")
else:
    users = db["users"]

In [8]:
users

ArangoDB collection name: users, id: 9320, type: document, status: loaded

In [9]:
all_users = df['sender'].append(df['receiver'])
all_users.head()

0    192614cd07e29f4f1f78da281ebeb7ef
1    e230a20a8a1bdfb8a4ef7376689a92ae
2    8ccccf41e5d1db2a9c0c85580ba452a3
3    177ad70e864b250986cd8cf9b6b445c0
4    b4fe7056bd4a9ba2bac7dbea137037db
dtype: object

In [10]:
all_unique_users = all_users.unique()
len(all_unique_users)

52852

In [11]:
for u in all_unique_users:
    new_doc = users.createDocument()
    new_doc._key = u
    new_doc.save()

In [12]:
sendTo = None
if not db.hasCollection(name='sendTo'):
    sendTo = db.createCollection(name='sendTo', className='Edges')
else:
    sendTo = db['sendTo']

In [13]:
transform_df = df.groupby(['sender', 'receiver']).agg({
    'amount': 'sum'
})
transform_df.reset_index(inplace=True)
transform_df

,sender,receiver,amount
0,00055b984cdfbe6b799d5debf8323c64,43026332e53541a19627d074c1833200,1500
1,0006544b05ceb936b9ea7b2af41e8f22,060c23c14d40d3e85d6054783db38a02,20000
2,000708e22b25c950d014c17e0424cbfb,042c9ebffdff74530d48a92d3a6b9ea3,500000
3,00085550d7aa4e4f322a79effc6cc3fc,a6238e443bc03bfc46b3f6799c16ae2c,1033143
4,000c1d6349b283a51bc071e16a8ecb7a,9b3b61b0e9f6a17ad23750e4a6ad24a2,300000
...,...,...,...
64965,ffec1eefeca1a9e380f9f3a22741fd9b,f13ac74bb76453f35de347cef75169c9,1000
64966,fff0f5b16357c2708abb99912d4d8554,9e7ea3b96c3e22434f965b0429e0ca5c,1150000
64967,fff241fdea9422a93d907951b4378848,4aa9b0142b29517ec2838b24ec530e6b,80000
64968,fff5789119479e8add8babf66940bead,e030eb74a91918ebbbf361dae7e60270,20000


In [14]:
for sender, receiver, amount in transform_df.values:
    new_edge = sendTo.createDocument({
        "_from": "users/" + sender,
        "_to": "users/" + receiver
    })
    new_edge.save()